## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                        dfy.reset_index(drop=True)

                        for index,curie_id in enumerate(dfy.Curie):
                            print(index,curie_id)
                            node_num = dfy.iloc[index][3]
                            query_id = curie_id
                            if np.isnan(dfy.iloc[index][2]) == True:
                                len_check = len(child_response['fields']['data']['message']['results'])
                                len_check = int(len_check)
                            else:
                                len_check = int(dfy.iloc[index][2])

                            #print(node_num, query_id, len_check)

                            locs = []
                            for x, val in enumerate(child_response['fields']['data']['message']['results']):
                                #print(val)

                                if x < len_check:

                                    if query_id in val['node_bindings'][node_num][0]['id']:
                                        locs.append(x)
                            if not locs:
                                check_result = f'False'
                                print(check_result)
                                #pass
                            else:
                                check_result = f'True'
                                print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                            dict3[curie_id] = check_result    

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + ' ' + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        else:
            dictionary[child['actor']['agent']] = 'Running'
            
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [4]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [5]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n00,include,NaN,NaN
9,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallM...,CHEBI:45783,100.0,n3,include,NaN,NaN


<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [6]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    #sleep(600)

                    #result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(600)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json
A.0_RHOBTB2_direct
https://arax.ncats.io/?source=ARS&id=45276b5a-b7dd-4b73-99be-8094f77e23db
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct_inverse.json
A.0_RHOBTB2_direct_inverse
https://arax.ncats.io/?source=ARS&id=9a63fc92-4a82-43f1-8201-457a6e90691d
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2_RHOBTB2_twohop.json
A.2_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=12d64583-958f-488f-8ebc-bba9b29399ae
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_expanded_RHOBTB2_twohop.json
A.2a_expanded_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=b5acdf47-f5d7-4acf-93bf-6caca9c100f1
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol.json
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
https://arax.ncats.io/?so

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
dict_workflows

{'A.0_RHOBTB2_direct': '45276b5a-b7dd-4b73-99be-8094f77e23db',
 'A.0_RHOBTB2_direct_inverse': '9a63fc92-4a82-43f1-8201-457a6e90691d',
 'A.2_RHOBTB2_twohop': '12d64583-958f-488f-8ebc-bba9b29399ae',
 'A.2a_expanded_RHOBTB2_twohop': 'b5acdf47-f5d7-4acf-93bf-6caca9c100f1',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': '69d7d6c6-0c7e-4b80-b556-3cda7276d1c1',
 'B.3b_DILI-two-hop-drug': '0c994267-bf3a-4565-a106-11cd67669f03',
 'B.1_DILI-three-hop': 'f15f5611-b713-4875-8cff-2d6401156b55',
 'B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone': 'd2de38d2-dcb4-4040-8d96-a34566cf5478',
 'B.3a_DILI-two-hop-chemical-entity': '3871ed4a-c4a7-4421-9092-5d55a3f58393',
 'B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone': '3ecbd5b0-8154-4926-be39-d095478e082a',
 'B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin': 'c1907487-b401-47d1-8acb-040d8c0a3b15',
 'B.2f_DILI-fourth-one-hop-from-CHEMBL.COMPOUND_CHEMBL50_Quercetin': '5d87353e-fd6a-4b7d-ab78-48d76fe53a6f',


In [8]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    

    result_status = retrieve_ars_results(val, name, check_sheet)

    workflow_result_messages[keys] = result_status
    
    sleep(600)

A.0_RHOBTB2_direct.json 45276b5a-b7dd-4b73-99be-8094f77e23db
Done
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
False
ara-aragorn Done 2
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
False
ara-arax Done 2
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
False
ara-bte Done 2
Error
ara-unsecret Error 0
Done
ara-explanatory Done 0
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
curie id: PUBCHEM.COMPOUND:2950270 : INCLUDED at postion N == [0] on n1
ara-improving Done 1
A.0_RHOBTB2_direct_inverse.json 9a63fc92-4a82-43f1-8201-457a6e90691d
Done
Done
ara-aragorn Done 4
Done
ara-arax Done 4
Done
ara-bte Done 1
Error
ara-unsecret Error 0
Done
ara-explanatory Done 19
Done
ara-improving ARS Error 0
A.2_RHOBTB2_twohop.json 12d64583-958f-488f-8ebc-bba9b29399ae
Done
Done
ara-aragorn Done 14661
Done
ara-arax Done 500
Done
ara-bte Done 4344
Error
ara-unsecret Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 1000
A.2a_expanded_RHOBTB2_twohop.json b5acdf47-f

In [9]:
workflow_result_messages

{'A.0_RHOBTB2_direct': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=45276b5a-b7dd-4b73-99be-8094f77e23db',
   'ara-aragorn': "Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'}",
   'ara-arax': "Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'}",
   'ara-bte': "Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'}",
   'ara-unsecret': 'Error: 400',
   'ara-explanatory': 'No Results: 200',
   'ara-improving': "Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'}"},
  {'ara-aragorn': {'infores:aragorn',
    'infores:aragorn-ranker-ara',
    'infores:text-mining-provider-targeted',
    'infores:translator-biothings-explorer'},
   'ara-arax': {'infores:arax',
    'infores:text-mining-provider-targeted',
    'infores:translator-biothings-explorer'},
   'ara-bte': {'infores:text-mining-provider-targeted',
    'infores:translator-biothings-explorer'},
   'ara-improving': {'infores:improving-agent',
    'infores:lincs',
    'infores:spoke'}}],
 'A.0_RHOBTB2_direct_inverse': [{'pk_id': 'https://

### Creating dataframe for workflows with PK

<br>

In [10]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)

    count = 0
    for key, value in workflow_result_messages[k][0].items():
        
        
        count= count+1
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
7
A.0_RHOBTB2_direct_inverse
7
A.2_RHOBTB2_twohop
7
A.2a_expanded_RHOBTB2_twohop
7
B.1_DILI-three-hop
7
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
7
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
7
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
7
B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
7
B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
7
B.2f_DILI-fourth-one-hop-from-CHEMBL.COMPOUND_CHEMBL50_Quercetin
7
B.3a_DILI-two-hop-chemical-entity
7
B.3b_DILI-two-hop-drug
7
C1
7
C2
7
C3
7
D.1_parkinsons-crohns
7
D.2_ssri-heart-disease
7
D.3_ssri-heart-disease-one-hop
7
D.4_tryptophan-kynurenine
7
D.6_metformin-ferritin
7


In [11]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [12]:
df

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=45276b5a-...,https://arax.ncats.io/?source=ARS&id=9a63fc92-...,https://arax.ncats.io/?source=ARS&id=12d64583-...,https://arax.ncats.io/?source=ARS&id=b5acdf47-...,https://arax.ncats.io/?source=ARS&id=f15f5611-...,https://arax.ncats.io/?source=ARS&id=d2de38d2-...,https://arax.ncats.io/?source=ARS&id=69d7d6c6-...,https://arax.ncats.io/?source=ARS&id=c1907487-...,https://arax.ncats.io/?source=ARS&id=3ecbd5b0-...,https://arax.ncats.io/?source=ARS&id=d7370cea-...,...,https://arax.ncats.io/?source=ARS&id=3871ed4a-...,https://arax.ncats.io/?source=ARS&id=0c994267-...,https://arax.ncats.io/?source=ARS&id=62e23d11-...,https://arax.ncats.io/?source=ARS&id=28a68b1c-...,https://arax.ncats.io/?source=ARS&id=db5779a2-...,https://arax.ncats.io/?source=ARS&id=947526be-...,https://arax.ncats.io/?source=ARS&id=e159af5a-...,https://arax.ncats.io/?source=ARS&id=574fbd0f-...,https://arax.ncats.io/?source=ARS&id=593b53fe-...,https://arax.ncats.io/?source=ARS&id=994c621d-...
ara-aragorn,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200 {},Results: 200 {},Results: 200 {},No Results: 200,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200 {},Results: 200 {},No Results: 200
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},"Results: 200 {'NCBIGene:120892': 'False', 'NCB...","Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200 {},No Results: 200,"Results: 200 {'UniProtKB:P54646': 'True', 'NCB..."
ara-bte,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200 {},Results: 200 {},Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200 {},No Results: 200,"Results: 200 {'UniProtKB:P54646': 'False', 'NC..."
ara-unsecret,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,...,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400
ara-explanatory,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,No Results: 200,No Results: 200,Results: 200 {},Error: 504,No Results: 200,"Results: 200 {'NCBIGene:120892': 'False', 'NCB...",No Results: 200,Results: 200 {},No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200 {},No Results: 200,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,No Results: 200,No Results: 200,Results: 200 {},No Results: 200,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200 {},Results: 200 {},"Results: 200 {'UniProtKB:

In [13]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [14]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.0_RHOBTB2_direct_inverse
A.2_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
B.1_DILI-three-hop
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.2f_DILI-fourth-one-hop-from-CHEMBL.COMPOUND_CHEMBL50_Quercetin
B.3a_DILI-two-hop-chemical-entity
B.3b_DILI-two-hop-drug
C1
C2
C3
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [15]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:text-mining-provider-targeted',
               'infores:translator-biothings-explorer'},
              'ara-arax': {'infores:arax',
               'infores:text-mining-provider-targeted',
               'infores:translator-biothings-explorer'},
              'ara-bte': {'infores:text-mining-provider-targeted',
               'infores:translator-biothings-explorer'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.0_RHOBTB2_direct_inverse': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-covidkop',
               'infores:automat-ctd',
               'infores:ctd',
               'infores:translator-biothings-explorer'},
              'ara-arax': {'infores:arax',
             

In [16]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [17]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-aragorn', 'infores:aragorn'],
 ['A.0_RHOBTB2_direct',
  'ara-aragorn',
  'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct',
  'ara-aragorn',
  'infores:text-mining-provider-targeted'],
 ['A.0_RHOBTB2_direct', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:arax'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:text-mining-provider-targeted'],
 ['A.0_RHOBTB2_direct', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct', 'ara-bte', 'infores:text-mining-provider-targeted'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:automat-ctd'],
 [

In [18]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [19]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [20]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [21]:
df2test = pd.DataFrame(df2test.unstack().T)

In [22]:
df2test

Workflow,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[ara-arax, ara-bte]",[ara-bte],[ara-bte],NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving]
infores:tcrd,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,[ara-arax]
infores:text-mining-provider-targeted,"[ara-aragorn, ara-arax, ara-bte]",NaN,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]",NaN,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",...,NaN,NaN,NaN,"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]",[ara-bte],NaN,"[ara-arax, ara-bte]"


In [23]:
#df2test.drop([''], axis=0, inplace=True)

In [24]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [25]:
df2test

Workflow,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[ara-arax, ara-bte]",[ara-bte],[ara-bte],NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving]
infores:tcrd,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,[ara-arax]
infores:text-mining-provider-targeted,"[ara-aragorn, ara-arax, ara-bte]",NaN,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]",NaN,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",...,NaN,NaN,NaN,"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]",[ara-bte],NaN,"[ara-arax, ara-bte]"


In [26]:
#df2.replace([], 'None', regex=True,inplace=True)

In [27]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [28]:
df2test = df2test.rename_axis(None)

In [29]:
df2test.columns.name = None

In [30]:
df2test.fillna('', inplace=True)

In [31]:
df2test

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
CTD,,,,,,,,,,,...,,,,,,[ara-bte],[ara-bte],,,
DisGeNET,,,,,,,,,,,...,,,,,"[ara-arax, ara-bte]",[ara-bte],[ara-bte],,,
DrugCentral Indication,,,,,,,,,,,...,,,,[ara-bte],,,[ara-bte],,,
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,,[ara-bte],,,,
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,[ara-bte],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,,,,,,,,,,,...,,,,,,,,,,[ara-improving]
infores:tcrd,,,[ara-aragorn],,,,,,,,...,,,,[ara-arax],,,,,,[ara-arax]
infores:text-mining-provider-targeted,"[ara-aragorn, ara-arax, ara-bte]",,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]",,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",...,,,,"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]",[ara-bte],,"[ara-arax, ara-bte]"
infores:translator-biothings-explorer,"[ara-aragorn, ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte, ara-explanatory]","[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]",[ara-arax],"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",...,[ara-arax],"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte, ara-explanatory]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-bte, ara-explanatory]",,"[ara-arax, ara-bte]"


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [32]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [33]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [34]:
infores_catalog = infores_catalog[:337]

In [35]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [36]:
dict_map

{'CTD': 'Illegal value',
 'DisGeNET': 'Illegal value',
 'DrugCentral Indication': 'Illegal value',
 'JensenLab Knowledge AmyCo': 'Illegal value',
 'JensenLab Knowledge GHR': 'Illegal value',
 'JensenLab Knowledge UniProtKB-KW': 'Illegal value',
 'JensenLab Text Mining': 'Illegal value',
 'eRAM': 'Illegal value',
 'infores:aeolus': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-cord19': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-gwas-catalog': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hgnc': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-human-goa': 'KP',
 'infores:automat-ontology-hierarchy': 'KP',
 'infores:automat-panther': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:automat-uberongraph': 

In [37]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [38]:
df2test

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog
CTD,,,,,,,,,,,...,,,,,[ara-bte],[ara-bte],,,,Illegal value
DisGeNET,,,,,,,,,,,...,,,,"[ara-arax, ara-bte]",[ara-bte],[ara-bte],,,,Illegal value
DrugCentral Indication,,,,,,,,,,,...,,,[ara-bte],,,[ara-bte],,,,Illegal value
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,[ara-bte],,,,,Illegal value
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,[ara-bte],,,,,Illegal value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,,,,,,,,,,,...,,,,,,,,,[ara-improving],External Source
infores:tcrd,,,[ara-aragorn],,,,,,,,...,,,[ara-arax],,,,,,[ara-arax],External Source
infores:text-mining-provider-targeted,"[ara-aragorn, ara-arax, ara-bte]",,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]",,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",...,,,"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]",[ara-bte],,"[ara-arax, ara-bte]",KP
infores:translator-biothings-explorer,"[ara-aragorn, ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte, ara-explanatory]","[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]",[ara-arax],"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",...,"[ara-arax, ara-bte]","[ara-aragorn, ara-arax, ara-bte, ara-explanatory]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-arax, ara-bte]","[ara-bte, ara-explanatory]",,"[ara-arax, ara-bte]",ARA


In [39]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [40]:
df2test

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:sri-ontology,,,,,,,,,,,...,[ara-explanatory],,,,,[ara-explanatory],,,KP,KP
infores:biothings-dgidb,,,[ara-bte],,,,,,,,...,,,,,"[ara-arax, ara-bte]",,,"[ara-arax, ara-bte]",KP,KP
infores:biothings-diseases,,,,,,,,,,,...,,[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,KP,KP
infores:biothings-idisk,,,,,,,,,,,...,,,,[ara-bte],,,,,KP,KP
infores:biothings-multiomics-biggim-drugresponse,,"[ara-arax, ara-explanatory]",[ara-arax],,,,,,,,...,,,,,,,,,KP,KP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,,,,,,,,,,,...,[ara-explanatory],,,[ara-bte],,"[ara-aragorn, ara-bte]",,,Illegal value,NaN
infores:hetio,,,"[ara-aragorn, ara-arax, ara-bte]",[ara-aragorn],,"[ara-arax, ara-bte, ara-explanatory]",[ara-aragorn],,"[ara-arax, ara-bte]","[ara-arax, ara-bte, ara-explanatory]",...,"[ara-aragorn, ara-explanatory]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]","[ara-aragorn, ara-bte]",[ara-aragorn],"[ara-arax, ara-bte]",Illegal value,NaN
infores:ontological-hierarchy,,,[ara-aragorn],[ara-aragorn],,,[ara-aragorn],,,,...,[ara-aragorn],,,[ara-bte],,[ara-aragorn],,[ara-bte],Illegal value,NaN
infores:pdsp,,,,,,,,,,,...,,,,,[ara-arax],,,,Illegal value,NaN


In [41]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [42]:
df2test

,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
infores:sri-ontology,KP,,,,,,,,,,...,,,[ara-explanatory],,,,,[ara-explanatory],,
infores:biothings-dgidb,KP,,,[ara-bte],,,,,,,...,,,,,,,"[ara-arax, ara-bte]",,,"[ara-arax, ara-bte]"
infores:biothings-diseases,KP,,,,,,,,,,...,,,,[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,
infores:biothings-idisk,KP,,,,,,,,,,...,,,,,,[ara-bte],,,,
infores:biothings-multiomics-biggim-drugresponse,KP,,"[ara-arax, ara-explanatory]",[ara-arax],,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,Illegal value,,,,,,,,,,...,,,[ara-explanatory],,,[ara-bte],,"[ara-aragorn, ara-bte]",,
infores:hetio,Illegal value,,,"[ara-aragorn, ara-arax, ara-bte]",[ara-aragorn],,"[ara-arax, ara-bte, ara-explanatory]",[ara-aragorn],,"[ara-arax, ara-bte]",...,[ara-explanatory],,"[ara-aragorn, ara-explanatory]","[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]","[ara-aragorn, ara-bte]",[ara-aragorn],"[ara-arax, ara-bte]"
infores:ontological-hierarchy,Illegal value,,,[ara-aragorn],[ara-aragorn],,,[ara-aragorn],,,...,,,[ara-aragorn],,,[ara-bte],,[ara-aragorn],,[ara-bte]
infores:pdsp,Illegal value,,,,,,,,,,...,,,,,,,[ara-arax],,,


In [43]:
#df2test['Query Type'] = 'Sync'

#df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [44]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [45]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [46]:
df.sort_index(inplace=True)

In [47]:
df

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,"Results: 200 {'NCBIGene:120892': 'False', 'NCB...","Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200,No Results: 200,"Results: 200 {'UniProtKB:P54646': 'True', 'NCB..."
ara-bte,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200,No Results: 200,"Results: 200 {'UniProtKB:P54646': 'False', 'NC..."
ara-explanatory,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,"Results: 200 {'NCBIGene:120892': 'False', 'NCB...",No Results: 200,Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200,Results: 200,"Results: 200 {'UniProtKB:P54646': 'False', 'NC..."
ara-unsecret,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,...,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400
pk,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...",...,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."


<br>

### Highlight the cells In Excel

<br>

In [48]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [49]:
styled = df.style.applymap(highlight)

In [50]:
styled

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.2f_DILI-fourth-one-hop-from-CHEMBL.COMPOUND_CHEMBL50_Quercetin,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C1,C2,C3,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'},Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'},Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,"Results: 200 {'NCBIGene:120892': 'False', 'NCBIGene:11315': 'False', 'NCBIGene:110357': 'False'}","Results: 200 {'NCBIGene:3988': 'False', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,No Results: 200,"Results: 200 {'UniProtKB:P54646': 'True', 'NCBIGene:5563': 'False', 'UniProtKB:P42345': 'False', 'UniProtKB:Q15166': 'False', 'UniProtKB:P16422': 'False', 'UniProtKB:Q9NQ76': 'False', 'UniProtKB:P18065': 'False', 'UniProtKB:P18510': 'False'}"
ara-bte,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'False'},Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,No Results: 200,"Results: 200 {'UniProtKB:P54646': 'False', 'NCBIGene:5563': 'True', 'UniProtKB:P42345': 'False', 'UniProtKB:Q15166': 'False', 'UniProtKB:P16422': 'False', 'UniProtKB:Q9NQ76': 'False', 'UniProtKB:P18065': 'False', 'UniProtKB:P18510': 'False'}"
ara-explanatory,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 504,No Results: 200,"Results: 200 {'NCBIGene:120892': 'False', 'NCBIGene:11315': 'False', 'NCBIGene:110357': 'False'}",No Results: 200,Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGene:5627': 'False', 'NCBIGene:7043': 'False'}",Results: 200,Results: 200,"Results: 200 {'UniProtKB:P54646': 'False', 'NCBIGene:5563': 'False', 'UniProtKB:P42345': 'False', 'UniProtKB:Q15166': 'False', 'UniProtKB:P16422': 'False', 'UniProtKB:Q9NQ76': 'False', 'UniProtKB:P18065': 'False', 'UniProtKB:P18510': 'False'}"
ara-unsecret,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400
pk,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&id=45276b5a-b7dd-4b73-99be-8094f77e23db"", ""https://arax.ncats.io/?source=ARS&id=45276b

In [51]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [52]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [53]:
#styled

In [54]:
#df['Query Type'] = 'Sync'

#df = df[['Query Type']+ list(df.columns[:-1])]

In [55]:
#df.reset_index(inplace=True)

In [56]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [57]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [58]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [59]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_12_09-08_44_50_AM' id:1084775988>

In [61]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_12_09-08_44_50_AM")

In [66]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_CONTAINS', ['Error: 501']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()


rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()


rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()


rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()


# Select a range
cell_list = sh.range('B8:AS8')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

set_frozen(sh, cols=1, rows=1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

### Push Dataframe 2

<br>

In [63]:
wks2 = 'edge_attribute_source_' + date

In [64]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_12_09-08_44_50_AM' id:258574146>

In [65]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_12_09-08_44_50_AM")
set_frozen(ws2, cols=1, rows =1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}